# SNe in host galaxies
Owner: Alex Kim <br>
Last Verified to Run: 2018-07-23

In [1]:
import numpy
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from astropy.coordinates import SkyCoord
import GCRCatalogs
import buzzard_stellar_mass
import itertools

In [3]:
controlTime = 1e-9

In [4]:
# load coadd catalog (for a single tract)
coadd_cat = GCRCatalogs.load_catalog('buzzard_v1.6_test')

In [5]:
kfile = "/global/projecta/projectdirs/lsst/groups/CS/Buzzard/metadata/templates/k_nmf_derived.default.fits"

gdq = buzzard_stellar_mass.GetDerivedQuantities(kfile)


coadd_cat.add_derived_quantity(
    "sfr",
    lambda z, *coeffs: gdq.sfr(numpy.vstack(coeffs).T, z),
    "redshift",
    "truth/COEFFS/0",
    "truth/COEFFS/1",
    "truth/COEFFS/2",
    "truth/COEFFS/3",
    "truth/COEFFS/4"
)

coadd_cat.add_derived_quantity(
    "smass",
    lambda z, *coeffs: gdq.smass(numpy.vstack(coeffs).T, z),
    "redshift",
    "truth/COEFFS/0",
    "truth/COEFFS/1",
    "truth/COEFFS/2",
    "truth/COEFFS/3",
    "truth/COEFFS/4"
)

coadd_cat.add_derived_quantity(
    "snrate_obs",
    lambda sfr, smass, z: 1.23e-03*sfr**(.95)+.26e-10*smass**(.72)/(1+z),
    "sfr",
    "smass",
    "redshift_true"
)

def gen_poisson(x, seed=123):
    mask = numpy.isfinite(x)
    out = numpy.random.RandomState(seed).poisson(numpy.where(mask, x, 0))
    out[~mask] = numpy.iinfo(numpy.int).max
    return out

coadd_cat.add_derived_quantity( "numSN", gen_poisson, "snrate_obs" )

In [6]:
quantities = ["numSN","redshift", "redshift_true","ra","dec","galaxy_id"]
filts = [
    'redshift < 0.01',
    (numpy.isfinite,"truth/COEFFS/0"),
    (numpy.isfinite,"truth/COEFFS/1"),
    (numpy.isfinite,"truth/COEFFS/2"),
    (numpy.isfinite,"truth/COEFFS/3"),
    (numpy.isfinite,"truth/COEFFS/4"),
    'numSN > 0'
    ]

In [7]:
for data in coadd_cat.get_quantities(quantities, filters = filts, return_iterator=True):
    print(data['numSN'])

(0,)
